In [97]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
# from linearmodels.panel import PanelOLS

In [25]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

## 2020 treatment vs control

In [88]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

wf["temp2"] = wf["temp"] ** 2
wf["l_aqi"] = np.log(1 + wf["aqi"])
wf["l_pm"] = np.log(1 + wf["pm"])
wf2020 = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna(
    subset = ['aqi', 'pm']
)
wf2020['cities'] = wf2020['city_code'].astype('category')
wf2020['days'] = wf2020['daynum'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

In [95]:
fixed = ['treat']
for col in wf2020.columns:
    if 'cities' in col or 'days' in col:
        fixed.append(col)
        
weather = ['prec', 'snow', 'temp', 'temp2']

In [96]:
out = ["aqi", "l_aqi", "pm", "l_pm"]

for Yname in out:
    Y = wf2020[Yname]
    X = wf2020[fixed + weather]
    lr = ()
    lr.fit(X, Y)
    print(lr.coef_[0])

-19.84407593228247
-0.1741297114238893
-14.069969671861866
-0.16783963935487584
